In [1]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoConfig, AutoModel

In [2]:
from dataset.dataset import T5Dataset
from dataset.preprocessor.preprocessors import T5BasicPreprocessor
from dataset.augmentation.augmentations import SimpleRandomUNK

In [3]:
MODEL_NAME = "KETI-AIR/ke-t5-small"
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

dataset = T5Dataset("/opt/ml/dataset/")
preprocessor = T5BasicPreprocessor()

dataset.set_tokenizer(tokenizer)
dataset.set_preprocessor(preprocessor)
dataset.preprocess()

Old: 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
New: klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 *비틀즈*가 1969년 앨범 《Abbey Road》에 담은 노래다.
Old: 호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
New: klue_re text: 호남이 기반인 바른미래당·#대안신당#·*민주평화당*이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
Old: K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
New: klue_re text: K리그2에서 성적 1위를 달리고 있는 *광주FC*는 지난 26일 #한국프로축구연맹#으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
Old: 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
New: klue_re text: 균일가 생활용품점 (주)*아성다이소*(대표 #박정부#)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
Old: 1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.
New: klue_re text: #1967#년 프로 야구 드래프트 1순위로 *요미우리 자이언츠*에게 입단하면서 등번호는 8번으로 배정되었다.


In [4]:
dataset.data.head()

,id,sentence,sentence_length,subject_entity_word,subject_entity_start_idx,subject_entity_end_idx,subject_entity_type,object_entity_word,object_entity_start_idx,object_entity_end_idx,object_entity_type,concat_entity,label,source,t5_inputs,t5_sbj_s_idx,t5_sbj_e_idx,t5_obj_s_idx,t5_obj_e_idx
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,59,비틀즈,24,26,ORG,조지 해리슨,13,18,PER,비틀즈[SEP]조지 해리슨,0,wikipedia,klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 *비틀즈*가...,40,45,27,35
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,54,민주평화당,19,23,ORG,대안신당,14,17,ORG,민주평화당[SEP]대안신당,0,wikitree,klue_re text: 호남이 기반인 바른미래당·#대안신당#·*민주평화당*이 우여...,35,42,28,34
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,99,광주FC,21,24,ORG,한국프로축구연맹,34,41,ORG,광주FC[SEP]한국프로축구연맹,20,wikitree,klue_re text: K리그2에서 성적 1위를 달리고 있는 *광주FC*는 지난 ...,35,41,50,60
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,78,아성다이소,13,17,ORG,박정부,22,24,PER,아성다이소[SEP]박정부,1,wikitree,klue_re text: 균일가 생활용품점 (주)*아성다이소*(대표 #박정부#)는 ...,27,34,38,43
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,56,요미우리 자이언츠,22,30,ORG,1967,0,3,DAT,요미우리 자이언츠[SEP]1967,0,wikipedia,klue_re text: #1967#년 프로 야구 드래프트 1순위로 *요미우리 자이...,38,49,14,20


In [5]:
idx = 0
sentence = dataset.data['t5_inputs'].iloc[0]
print(sentence)

klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 *비틀즈*가 1969년 앨범 《Abbey Road》에 담은 노래다.


In [6]:
sbj_s_idx = dataset.data['t5_sbj_s_idx'].iloc[0]
sbj_e_idx = dataset.data['t5_sbj_e_idx'].iloc[0]
print(sbj_s_idx, sbj_e_idx)

40 45


In [7]:
sentence[:sbj_s_idx]

'klue_re text: 〈Something〉는 #조지 해리슨#이 쓰고 '

In [10]:
obj_s_idx = dataset.data['t5_obj_s_idx'].iloc[0]
obj_e_idx = dataset.data['t5_obj_e_idx'].iloc[0]
print(sbj_s_idx, sbj_e_idx)

27 35


In [11]:
sentence[:obj_s_idx]

'klue_re text: 〈Something〉는 '